[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/amrsalah944/DeepGeo/blob/main/google_maps_route_display.ipynb)

## **Step 1: Create a Google Cloud Project**

1. Go to Google Cloud Console.
    
2. Click on **Select a project** → **New Project**.
    
3. Enter a **Project Name** (e.g., "Route Mapping App") and click **Create**.
    
4. Wait for the project to be created, then select it.
    

---

## **Step 2: Enable the Required APIs**

To fetch route data, enable the following APIs:
    
- **Google Directions API** (for getting route details)
    
- **Google Places API** (optional, for autocomplete suggestions)
    

### **To enable APIs:**

1. Go to Google Cloud Console APIs & Services.
    
2. Search for:
    
    - **Google Maps JavaScript API** → Click **Enable**
        
    - **Google Directions API** → Click **Enable**
        
    - (Optional) **Google Places API** → Click **Enable**
        

---

## **Step 3: Generate an API Key**

1. Go to Google Cloud Credentials.
    
2. Click **Create Credentials** → **API Key**.
    
3. Copy the generated API key and add it to your .env file or use it as a variable in your code "YOUR-API-KEY"

!pip install streamlit folium streamlit-folium requests polyline

In [6]:
!pip install streamlit folium streamlit-folium requests polyline python-dotenv

In [ ]:
import requests
import polyline
import folium
from IPython.display import display, HTML
from dotenv import load_dotenv
import os
from pathlib import Path

# Check if .env file exists and load it
env_path = Path('.env')
if env_path.exists():
    load_dotenv()
    api_key = os.getenv('GOOGLE_MAPS_API_KEY')
else:
    # Fallback to hardcoded API key if .env doesn't exist
    print("Note: .env file not found. Using fallback API key.")
    api_key = "YOUR-API-KEY"  # Your API key here

def get_route_points(origin, destination):
    """
    Get the points along a route using the Google Directions API.
    :param origin: Starting point as "latitude,longitude"
    :param destination: Ending point as "latitude,longitude"
    :return: List of (latitude, longitude) points
    """
    if not api_key:
        raise ValueError("No API key available")

    url = "https://maps.googleapis.com/maps/api/directions/json"
    params = {
        "origin": origin,
        "destination": destination,
        "key": api_key
    }
    response = requests.get(url, params=params)
    data = response.json()

    if data['status'] == 'OK':
        polyline_str = data['routes'][0]['overview_polyline']['points']
        points = polyline.decode(polyline_str)
        print("Route points:", points)
        return points
    else:
        print("Error:", data.get("error_message", "Unknown error"))
        return None

def folium_deepnote_show(m):
    data = m.get_root().render()
    data_fixed_height = data.replace('width: 100%;height: 100%', 'width: 100%').replace('height: 100.0%;', 'height: 609px;', 1)
    display(HTML(data_fixed_height))
            
    
# Example usage
origin = "29.1354505,30.2304959"
destination = "29.1503267,30.2603045"

try:
    route_points = get_route_points(origin, destination)

    if route_points:
        # Create a map centered at the first point
        m = folium.Map(location=route_points[0], zoom_start=14)

        # Add the route as a polyline
        folium.PolyLine(route_points, color="blue", weight=5, opacity=0.8).add_to(m)

        # Add markers for origin and destination
        folium.Marker(location=route_points[0], popup="Origin", icon=folium.Icon(color="green")).add_to(m)
        folium.Marker(location=route_points[-1], popup="Destination", icon=folium.Icon(color="red")).add_to(m)

        # Display the map using the custom function
        folium_deepnote_show(m)
    else:
        print("Failed to fetch route points. Please check the coordinates or API key.")
except Exception as e:
    print(f"An error occurred: {str(e)}")